In [1]:
#Initial Imports
import requests
import pandas as pd
import time

In [ ]:
#Making the initial call for the summoner profile and parsing the account ID from it
def getSummonerID(ign, api):
        url = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/"+ign+"?api_key="+api
        response = requests.get(url).json()
        return response['accountId']

In [ ]:
#Returns an array of the last 50 match IDs the user had played within the given queue parameters
def getMatchData(accountID,api):
  #Queues included in here are only those that exclude co-op vs AI to not let the stats get too far skewed
    url = "https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/"+accountID+"?queue=400&queue=420&queue=430&queue=440&endIndex=5&api_key=" + api
    response = requests.get(url).json()
    return [x['gameId'] for x in response['matches']]

In [ ]:
#establishes the indexing for the user in each json's set of match statistics.
def getParticipantID(matchStats, ign):
    #Each match has exactly 10 players
    for i in range(10):
        tempIGN = matchStats['participantIdentities'][i]['player']['summonerName']
        if tempIGN == ign:
            participantID = i
    return participantID

In [ ]:
#parses out the kills, deaths and assists values for each match and returns a ratio for those 3 values.
def calculateKDA(matchStats, participantID):
    #Traditional KDA doesn't differentiate between 0 deaths and 1 death for ratio calculations. This handles the divide by 0 edge case.
    if matchStats['participants'][participantID]['stats']['deaths'] == 0:
        deaths = 1
    else:
        deaths = matchStats['participants'][participantID]['stats']['deaths']

    kills = matchStats['participants'][participantID]['stats']['kills']
    assists = matchStats['participants'][participantID]['stats']['assists']
    kda = (kills + assists)/deaths
    return kda

In [ ]:
#Returns the vision score for the player per match. Note that this value is heavily correlated with the amount of wards bought, but is not equivilant to it.
def calculateVisionScore(matchStats, participantID):
    visionScore = matchStats['participants'][participantID]['stats']['visionScore']
    return visionScore

In [ ]:
#Returns the time spent applying crown control effects to enemy champions.
def calculateCCTime(matchStats, participantID):
    ccTime = matchStats['participants'][participantID]['stats']['totalTimeCrowdControlDealt']
    return ccTime

In [ ]:
#Returns the role that the player played. The only value of interest here is "TOP". All other values will pass and not be a part of the calculations.
def getLaneStats(matchStats, participantID):
    lane = matchStats['participants'][participantID]['timeline']['lane']
    return lane

In [ ]:
#Returns the CSD prior to 10 minutes in each match's time line.
def getCSD(matchStats, participantID, lane):
    #The try-except block is implemented here to check for the cases where CSD is unable to be calculated by Riot's API.
    try:
        #Supports don't traditionally CS, any differentials here are omitted since this is not a metric that defines proficiency at the role.
        if lane != "SUPPORT":
            csd = matchStats['participants'][participantID]['timeline']['csDiffPerMinDeltas']['0-10']
            return csd
    except:
        return False 

In [ ]:
def getWards(matchStats, participantID):
	wards = matchStats['participants'][participantID]['stats']['visionWardsBoughtInGame']
	return wards

In [ ]:
def getIGNs(matchData, ign, api):
	igns = set()
	for match in matchData:
		url = "https://na1.api.riotgames.com/lol/match/v4/matches/"+ str(match) +"?api_key="+ api
		response = requests.get(url).json()
		for res in response["participantIdentities"]:
			igns.add(res['player']['summonerName'])


	return igns